In [1]:
import numpy as np

In [2]:
class Variable:
    def __init__(self, data: np.ndarray) -> None:
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        # 再帰 → while
        # メモリや拡張性の観点で優位
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()  # 1. Get a function
            x, y = f.input, f.output  # 2. Get the function's input/output
            x.grad = f.backward(y.grad)  # 3. Call the function's backward

            if x.creator is not None:
                funcs.append(x.creator)

In [3]:
class Function:
    def __call__(self, input: Variable) -> Variable:
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [4]:
class Square(Function):
    def forward(self, x):
        y = x**2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [5]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [6]:
A = Square()
B = Exp()
C = Square()

In [7]:
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [8]:
# backward
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256
